In [ ]:
import requests
from bs4 import BeautifulSoup
import argparse
import os
import pandas as pd
import xlrd

In [ ]:
dirName = 'images_dataset_final/'
last_pagination = 4

if not os.path.exists(dirName):
    os.mkdir(dirName)
cwd = os.getcwd()

headers_std = {
'User-Agent': 
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
'Content-Type': 'text/html',
}

In [ ]:
inp = pd.read_csv('queries.csv')
item_cat = inp.item_category

for i in range(len(item_cat)):
    item_category = item_cat[i]
    query = item_category.replace(' ','+')

    link  = "https://www.amazon.in/s?k="+query+"&ref=nb_sb_noss"
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'lxml')
    
    urls = []

    for x in range (1, last_pagination):
        urls.append(link + '&page=' + str(x))

    img_urls = list()
    page_urls = list()
    img_names = list()
    names = list()

    os.chdir(dirName)
    if not os.path.exists(item_category):
        os.mkdir(item_category+'/')
    os.chdir(item_category+'/')

    k = 1 
    pag = 1
    for x in urls:
        print('Pagination : ' + str(pag))
        req = requests.get(x,headers=headers_std).text
        soup = BeautifulSoup(req, 'lxml')
        imgs = soup.find_all('img',{'class':'s-image'})
        pageurls = soup.find_all('a',{'class':"a-link-normal s-no-outline"})

        for (pg,i) in enumerate(imgs):
            if str(i).find('src') != -1:
                try:
                    page = 'https://amazon.in' + pageurls[pg]['href']
                except IndexError:
                    page = 'null'
                
                if(page=='null'):
                    break
                    
                url = i['src']
                name = i['alt']
                names.append(name)
                img_urls.append(url)
                
                page_urls.append(page)
                
                image_name = url.split('.')[-3].split('/')[-1]
                img_names.append(image_name)
                name_image_folder = image_name + '.jpg'
                image = requests.get(url).content
                
                with open(name_image_folder, 'wb') as handler:
                    handler.write(image)
                
            k += 1
        pag += 1
        
    os.chdir(cwd)  

    try:
        df1 = pd.read_csv('amazon_data.csv')
    except:
        df1 = pd.DataFrame()

    df2 = pd.DataFrame({'image_url' : img_urls, 'image_id': img_names, 'prod_name':names, 'page_link': page_urls})


    df = pd.concat([df1, df2])

    df.to_csv('amazon_data.csv',index=False)
    print(item_category+" finished!")